In [1]:
import pandas as pd
import json
import random

Preparing dataset

In [107]:
df = pd.read_excel("FR_NFR_Dataset.xlsx", dtype=str)

In [108]:
df['Type'] = df['Type'].apply(lambda x: x.strip() if isinstance(x, str) else "Unknown")
if df['Type'].isna().sum() > 0:
    print("NaN values found after processing:", df[df['Type'].isna()])

In [109]:
jsonl_data = []

In [110]:
for index, row in df.iterrows():
    jsonl_data.append({
        "messages": [
            {"role": "user", "content": row["Requirement Text"]},
            {"role": "assistant", "content": row["Type"]}
        ]
    })

In [111]:
with open("ready_data.jsonl", "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")

In [112]:
file_path = "ready_data.jsonl"
with open(file_path, "r") as f:
    data = [json.loads(line) for line in f]
    
random.shuffle(data)

split_index = int(0.8 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

with open("train_data.jsonl", "w") as train_file:
    for entry in train_data:
        train_file.write(json.dumps(entry) + "\n")

with open("test_data.jsonl", "w") as test_file:
    for entry in test_data:
        test_file.write(json.dumps(entry) + "\n")

Fine-tuning GPT-4o-mini

In [9]:
from openai import OpenAI

with open("../openai_api_key.txt", "r") as file:
        openai_api_key = file.read().strip()
        client = OpenAI(api_key=openai_api_key)

In [114]:
file_response = client.files.create(
    file=open("train_data.jsonl", "rb"), purpose="fine-tune"
)
file_obj_id = file_response.id
print(f"Uploaded file! Object id: {file_obj_id}")

Uploaded file! Object id: file-RoCN0gaeYupI7arTfi7F2hHy


In [115]:
fine_tune = client.fine_tuning.jobs.create(training_file=file_obj_id, model="gpt-4o-mini-2024-07-18")
print(fine_tune)
fine_tune_id = fine_tune.id
print(f"Fine_tune ID: {fine_tune_id}")

FineTuningJob(id='ftjob-zYyte5ZCjVsiqCmSXDfneIZ8', created_at=1730592588, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-dOwXPJQ9qE4xluNhMVuFywBO', result_files=[], seed=1144410554, status='validating_files', trained_tokens=None, training_file='file-RoCN0gaeYupI7arTfi7F2hHy', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine_tune ID: ftjob-zYyte5ZCjVsiqCmSXDfneIZ8


In [116]:
# List 10 fine-tuning jobs
#client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(fine_tune_id)

FineTuningJob(id='ftjob-zYyte5ZCjVsiqCmSXDfneIZ8', created_at=1730592588, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-dOwXPJQ9qE4xluNhMVuFywBO', result_files=[], seed=1144410554, status='validating_files', trained_tokens=None, training_file='file-RoCN0gaeYupI7arTfi7F2hHy', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [118]:
completion = client.chat.completions.create(
    model=f"ft:gpt-4o-mini-2024-07-18:personal::APIxXqIF",
    messages=[{"role": "user", "content": "Classify this requirement: 'The system shall log all transactions.'"}]

)
print(completion.choices[0].message)

ChatCompletionMessage(content='FR', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


VALIDATION

In [119]:
with open("test_data.jsonl", "rb") as f:
    test_data = [json.loads(line) for line in f]

In [124]:
actual_labels = []
predicted_labels = []

for entry in test_data:
    message = entry["messages"][0]["content"]
    label = entry["messages"][1]["content"]
    
    actual_labels.append(label)
    
    completion = client.chat.completions.create(
        model=f"ft:gpt-4o-mini-2024-07-18:personal::APIxXqIF",
        messages=[{"role": "user", "content": message}]
    )
    
    predicted_label = completion.choices[0].message.content.strip()
    print(predicted_label)
    predicted_labels.append(predicted_label)

FR
FR
FR
NFR
FR
NFR
FR
FR
FR
FR
FR
FR
FR
FR
FR
FR
FR
NFR
FR
NFR
FR
FR
FR
FR
NFR
FR
FR
NFR
FR
FR
FR
NFR
NFR
FR
NFR
FR
FR
NFR
FR
NFR
FR
FR
FR
FR
FR
FR
FR
FR
NFR
NFR
FR
FR
FR
FR
FR
FR
FR
NFR
FR
FR
FR
FR
NFR
FR
NFR
NFR
FR
NFR
FR
NFR
NFR
NFR
FR
FR
FR
NFR
NFR
FR
FR
FR
FR
NFR
FR
NFR
FR
FR
NFR
FR
FR
FR
FR
FR
NFR
FR
FR
FR
FR
NFR
NFR
FR
NFR
FR
FR
FR
NFR
FR
FR
FR
NFR
FR
NFR
FR
FR
NFR
NFR
FR
NFR
NFR
FR
FR
FR
FR
FR
FR
FR
FR
NFR
FR
FR
FR
NFR
FR
NFR
FR
NFR
NFR
NFR
NFR
FR
FR
FR
FR
FR
NFR
NFR
NFR
FR
FR
FR
FR
NFR
FR
FR
FR
FR
FR
NFR
FR
FR
NFR
NFR
NFR
NFR
FR
NFR
FR
FR
FR
FR
NFR
NFR
FR
NFR
NFR
NFR
FR
NFR
NFR
FR
FR
FR
FR
FR
NFR
FR
FR
NFR
NFR
FR
Unknown
FR
NFR
FR
FR
FR
NFR
FR
NFR
FR
NFR
FR
FR
FR
FR
NFR
FR
FR
FR
FR
NFR
FR
FR
FR
FR
FR
NFR
FR
NFR
FR
FR
FR
FR
NFR
NFR
FR
FR
FR
FR
FR
NFR
FR
FR
FR
NFR
FR
NFR
FR
FR
FR
FR
FR
FR
FR
NFR
NFR
NFR
FR
FR
FR
FR
FR
NFR
NFR
NFR
NFR
FR
NFR
NFR
FR
FR
FR
FR
NFR
FR
NFR
NFR
FR
NFR
FR
FR
NFR
FR
NFR
FR
FR
FR
FR
FR
FR
NFR
NFR
FR
NFR
FR
FR
FR
FR
FR
NFR
FR
FR
FR
FR
NFR
NFR
FR
FR
NFR
FR

In [128]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels, average="weighted")
f1 = f1_score(actual_labels, predicted_labels, average="weighted")
recall = recall_score(actual_labels, predicted_labels, average="weighted")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.886437908496732
Precision: 0.8876014470154849
Recall: 0.886437908496732
F1 Score: 0.88456520820293


/Users/poonkuzhali.saran/PycharmProjects/GRE2/myvenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


GENERATING LABELS FOR MEDICAL REQ DATASET

In [2]:
df = pd.read_csv("medical_req_dataset.csv")

In [3]:
from helper import generate_label, get_openai_client

model_name = "ft:gpt-4o-mini-2024-07-18:personal::APIxXqIF"
client = get_openai_client()

for index, row in df.iterrows():
    text = row["Requirement Text"]
    gen_label = generate_label(model_name, client, text)
    df.at[index, "Generated_Label"] = gen_label

df.to_csv("fr_nfr_labeled_dataset.csv", index=False)

In [4]:
true_labels = df['Label']
generated_labels = df['Generated_Label']

In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
accuracy = accuracy_score(true_labels, generated_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, generated_labels, average="weighted")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

print("\nClassification Report:")
print(classification_report(true_labels, generated_labels))

Accuracy: 0.95
Precision: 0.9506566604127581
Recall: 0.95
F1 Score: 0.9499941403961092

Classification Report:
              precision    recall  f1-score   support

          FR       0.93      0.97      0.95       158
         NFR       0.97      0.93      0.95       162

    accuracy                           0.95       320
   macro avg       0.95      0.95      0.95       320
weighted avg       0.95      0.95      0.95       320
